# Random Forest Training with MLflow Model Registry

This notebook demonstrates how to:
1. Train Random Forest models with different hyperparameters
2. Log experiments to MLflow
3. Register the best model to the Model Registry
4. Set model aliases for versioning

## Setup and Imports

In [2]:
import os
import mlflow
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error, r2_score
from taxi_ride.data.preprocess_data import load_pickle, get_project_paths
import pandas as pd
from datetime import datetime 

## Configuration

In [3]:
# Experiment configuration
EXPERIMENT_NAME = "random-forest-experiments"
MODEL_NAME = "random-forest-taxi-model"

# MLflow setup
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.sklearn.autolog(log_input_examples=False, log_model_signatures=False)

# Initialize MLflow client
client = MlflowClient()

# Get data paths
paths = get_project_paths()
DATA_PATH = paths["PROCESSED_DATA_DIR"]

print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Data Path: {DATA_PATH}")

MLflow Tracking URI: http://127.0.0.1:5000
Experiment: random-forest-experiments
Data Path: /home/lisanab/fujitsu_laptop_files/MLOps/cookiecutter/cookiecutter-ml-course/data/processed


## Load Data

In [4]:
# Load datasets
X_train, y_train = load_pickle(os.path.join(DATA_PATH, "train.pkl"))
X_val, y_val = load_pickle(os.path.join(DATA_PATH, "val.pkl"))
X_test, y_test = load_pickle(os.path.join(DATA_PATH, "test.pkl"))

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

Training set: (63119, 6025)
Validation set: (61105, 6025)
Test set: (61119, 6025)


## Training Function

In [7]:
from taxi_ride.models.train_model import train_and_log_model, get_project_paths

paths = get_project_paths()
data_path = paths["PROCESSED_DATA_DIR"]

params = {
    "max_depth": 15,
    "n_estimators": 200,
    "min_samples_split": 4,
    "min_samples_leaf": 2,
    "random_state": 42
}

run_id, val_rmse, test_rmse, val_r2, test_r2 = train_and_log_model(data_path, params)

print("Finished training!")
print(f"Run ID: {run_id}")
print(f"Validation R2: {val_r2:.4f}, Test R2: {test_r2:.4f}")

2025/11/21 14:17:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run ID: 94f431ac7543455ba23260978a1c5513
Run name: 2025-11-21_14-17-02
Logged model with params {'max_depth': 15, 'n_estimators': 200, 'min_samples_split': 4, 'min_samples_leaf': 2, 'random_state': 42}
Validation R2: 0.6294, Test R2: 0.6445
🏃 View run 2025-11-21_14-17-02 at: http://127.0.0.1:5000/#/experiments/1/runs/94f431ac7543455ba23260978a1c5513
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
Finished training!
Run ID: 94f431ac7543455ba23260978a1c5513
Validation R2: 0.6294, Test R2: 0.6445


## Register Best Model to Model Registry

In [8]:
# Get the model URI from the best run
model_uri = f"runs:/{run_id}/model"

# Register the model
model_version = mlflow.register_model(
    model_uri=model_uri,
    name=MODEL_NAME
)

print(f"Model registered: {MODEL_NAME}")
print(f"Version: {model_version.version}")
print(f"Run ID: {run_id}")

Registered model 'random-forest-taxi-model' already exists. Creating a new version of this model...
2025/11/21 14:17:49 WARNING mlflow.tracking._model_registry.fluent: Run with id 94f431ac7543455ba23260978a1c5513 has no artifacts at artifact path 'model', registering model based on models:/m-0ece84f3f54b4d3787989a4c99f9a166 instead
2025/11/21 14:17:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest-taxi-model, version 5


Model registered: random-forest-taxi-model
Version: 5
Run ID: 94f431ac7543455ba23260978a1c5513


Created version '5' of model 'random-forest-taxi-model'.


## Set Model Aliases

In [5]:
# Set aliases for the registered model version
client.set_registered_model_alias(
    name=MODEL_NAME,
    alias="champion",
    version=1
)

client.set_registered_model_alias(
    name=MODEL_NAME,
    alias="production",
    version=2
)

print(f"Aliases set for version {model_version.version}:")
print("  - champion")
print("  - production")

NameError: name 'model_version' is not defined

## Update Model Description

In [9]:
# Add description to the model version
client.update_model_version(
    name=MODEL_NAME,
    version=model_version.version,
    description=f"Random Forest model trained on taxi ride data. "
                f"Validation R2: {val_r2:.4f}, "
                f"Test R2: {test_r2:.4f}"
)

print("Model description updated")

Model description updated


## Load Model from Registry

In [10]:
# Load model using alias
loaded_model = mlflow.sklearn.load_model(f"models:/{MODEL_NAME}@champion")

# Test the loaded model
test_predictions = loaded_model.predict(X_test)
test_r2 = r2_score(y_test, test_predictions)
test_rmse = root_mean_squared_error(y_test, test_predictions)

print(f"Loaded model from registry (champion alias)")
print(f"Test R2: {test_r2:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

Loaded model from registry (champion alias)
Test R2: 0.6338
Test RMSE: 6.0140


## View All Registered Models

In [11]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
MODEL_NAME = "random-forest-taxi-model"

model_versions = client.search_model_versions(f"name='{MODEL_NAME}'")
for mv in model_versions:
    # aliases is already a list of strings
    aliases = client.get_model_version(name=MODEL_NAME, version=mv.version).aliases
    print(f"Version: {mv.version}")
    print(f"  Run ID: {mv.run_id}")
    print(f"  Aliases: {aliases}")
    print(f"  Status: {mv.status}")
    print(f"  Description: {mv.description}")

Version: 5
  Run ID: 94f431ac7543455ba23260978a1c5513
  Aliases: []
  Status: READY
  Description: Random Forest model trained on taxi ride data. Validation R2: 0.6294, Test R2: 0.6445
Version: 4
  Run ID: 9eb1d49bffd543fdad7635ad75c32ff8
  Aliases: []
  Status: READY
  Description: 
Version: 3
  Run ID: a11e311030a945a28855b23c05f85ee4
  Aliases: []
  Status: READY
  Description: Random Forest model trained on taxi ride data. Validation R2: 0.6203, Test R2: 0.6367
Version: 2
  Run ID: 110105fb241a4116aaeb872bc9393922
  Aliases: ['production']
  Status: READY
  Description: 
Version: 1
  Run ID: 110105fb241a4116aaeb872bc9393922
  Aliases: ['champion']
  Status: READY
  Description: 


## Summary

This notebook demonstrated:
1. ✅ Training multiple Random Forest models with different hyperparameters
2. ✅ Logging experiments to MLflow
3. ✅ Comparing model performance
4. ✅ Registering the best model to the Model Registry
5. ✅ Setting model aliases (champion, production)
6. ✅ Loading models from the registry using aliases

You can now access your model in production using:
```python
model = mlflow.sklearn.load_model(f"models:/{MODEL_NAME}@champion")
```